Rapport : Quantitative Structure-Activity Relationship
Equipe : 

## **1 - Réprésentation des données**

In [162]:
# Importation des librairies
import pandas as pd
import plotly.express as px
from sklearn import linear_model
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import plotly.graph_objects as go
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


#### **a) Analyse de l'ensemble des données**
#### Analyse des critères statistiques des attributs

In [163]:
# Lecture des données du xlsx
data_file = "QSAR_dataset.xlsx"
# Stockage des données dans un dataframe
data = pd.read_excel(data_file,index_col=0)
print(data.shape)

(154, 75)


In [164]:
# Statistiques descriptives des 74 attributs
data.describe()

,apol,ASA+,ASA-,a_count,a_donacc,a_heavy,a_hyd,a_IC,a_nC,a_nCl,...,VSA,vsa_acc,vsa_hyd,vsa_pol,vsurf_A,vsurf_R,vsurf_S,vsurf_V,Weight,zagreb
count,154.000000,152.000000,153.000000,154.000000,154.000000,153.000000,154.000000,153.000000,154.000000,154.000000,...,154.000000,154.000000,154.000000,154.000000,154.000000,1.470000e+02,141.000000,136.000000,154.000000,154.000000
mean,34.610698,105.781739,359.928668,23.909091,0.292208,18.875817,17.350649,33.912102,11.649351,3.110390,...,273.307303,8.076532,239.944812,9.086768,2.379611,-6.802721e+08,-66.497364,-2.501405,359.813016,101.350649
std,5.951534,62.391286,111.225998,4.895461,0.862625,5.596428,5.028718,9.714722,2.472152,2.954031,...,52.783753,14.721655,59.915749,15.129738,2.637952,8.247861e+09,73.647379,2.807324,132.955027,33.487395
min,17.148172,8.778115,122.917570,12.000000,0.000000,10.000000,6.000000,12.000000,6.000000,0.000000,...,140.102050,0.000000,67.651054,0.000000,0.011998,-1.000000e+11,-209.769584,-8.247237,128.174000,46.000000
25%,31.534723,70.909811,330.864750,22.000000,0.000000,17.000000,16.000000,30.541887,12.000000,0.000000,...,246.182178,0.000000,203.302167,0.000000,0.124434,2.696068e-01,-132.566487,-5.058933,291.992000,88.000000
50%,35.579689,98.659012,389.503510,22.000000,0.000000,18.000000,17.000000,31.868664,12.000000,4.000000,...,281.160615,0.000000,253.968020,0.000000,0.376156,8.136986e-01,-10.648449,-0.411358,360.881990,94.000000
75%,38.401845,139.629990,427.294460,25.750000,0.000000,19.000000,18.000000,37.087944,12.000000,6.000000,...,295.503230,13.566921,272.261230,13.566921,4.786711,9.972196e+00,-3.509363,-0.133136,410.317990,106.000000
max,52.422001,356.764860,622.904600,43.000000,4.000000,43.000000,40.000000,86.319427,20.000000,10.000000,...,432.120120,59.150364,475.687620,59.150364,7.429943,1.611555e+01,-0.338738,-0.013318,959.170960,246.000000


In [165]:
#Il faut éléminer les outliers


#### Prétraitement des données
**Traitement des données manquantes** \
Nous avons décidé de procéder au traitement des données manquantes par imputation, au lieu de simplement supprimer les dites données. Les objets n'étant pas nombreux, il y a un risque d'obtenir des résultats faussés avec ce second choix.

In [166]:
# Détermination du type, du nombre et du pourcentage de valeurs manquantes par attribut
nb_m = data.isnull().sum().sort_values()
ratio_m = (data.isnull().sum()/data.shape[0]).sort_values()

In [167]:
manquant = pd.concat([nb_m, ratio_m], axis=1, sort=False)

In [168]:
# Affichage de ces données
df_manquants = pd.DataFrame({'Types': data[list(manquant.index.values)].dtypes,
                       'Nb manquants': nb_m,
                      '% de manquants': ratio_m,})
# On ne se concentre que sur les attributs aux valeurs manquantes
df_ADM = df_manquants[df_manquants["Nb manquants"]>0]
print(df_ADM)

           Types  Nb manquants  % de manquants
a_IC     float64             1        0.006494
a_heavy  float64             1        0.006494
ASA-     float64             1        0.006494
ASA+     float64             2        0.012987
vsurf_R  float64             7        0.045455
vsurf_S  float64            13        0.084416
vsurf_V  float64            18        0.116883


In [169]:
# Ajouter le ratio total et le nombre total de manquants faire une petite analyse dessus

Tout d'abord, on observe que tous les attributs manquants sont numériques. De plus, on remarque que la proportion de données manquantes est différente pour chaque attribut, on n'est donc pas dans le cas du MMCA (Données manquantes de Manière Complètement Aléatoire)<sup>**1**</sup>. On considère que nos données sont dans le cas MA (Manquantes Aléatoirement), car c'est la situation la plus courante<sup>**2**</sup>. Pour traiter les données MA il y a deux possibilités qui s'offrent à nous<sup>**3**</sup> :
- Imputation par régression linéaire
- Imputation multiple (On choisit kNN car c'est un algorithme qu'on a étudié en cours)

Nous avons choisi d'opter pour l'imputation kNN, car celle-ci peut effectuer une imputation plus simplement même s'il y a des valeurs NaN dans le Dataframe.
Afin de procéder à l'imputation des valeurs manquantes de ces attributs, il faut trouver ceux qui leur sont fortement corrélés. Pour éviter de se trouver dans une impasse, nous avons décidé d'ignorer, lors de l'identification des attributs corrélés à un ADM, les autres ADM.

Nous allons d'abord procéder à une normalisation des données pour éviter des biais systémiques<sup>**5**</sup>, et on procède à un "one-hot encoding" pour transformer l'attribut "Class" en attribut numérique et permettre l'imputation. 

In [170]:
# Liste des attributs avec des données manquantes
missing_attributes = ["a_IC","a_heavy","ASA-","ASA+","vsurf_R","vsurf_S","vsurf_V"]

# One hot encoding pour l'attribut Class
data = pd.get_dummies(data=data, columns=["Class"])

# Normalisation des données
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data), columns = data.columns)

**Imputation KNN (Optimisée)** 
\

Pour l'imputation multiple, nous allons procéder à une imputation kNN avec KNNImputer, puis optimiser notre modèle grâce à RandomForestRegressor(), qui permet de prédire des valeurs numériques. Elle permet de réduire le surapprentissage et d'améliorer la précision.https://www.kaggle.com/code/sangyunkang/knn-imputation
Pour déterminer le nombre de voisins k à prendre en compte, nous avons décidé d'utiliser une fonction, et de mesurer le taux d'erreur avec MSE pour plusieurs valeurs de k, ce qui nous permettre de nous décider.  (https://ai.plainenglish.io/understanding-common-regression-evaluation-metrics-mae-mse-rmse-r2-and-adjusted-r2-6c5709e614c4)MSE and RMSE are appropriate when larger errors need to be penalized, provided outliers are managed.

In [171]:
# Le but de cette fonction est de permettre l'évaluation du meilleur nombre de voisins pour l'imputation kNN
def eval_kNNimputation(attribute):
        res = []
        for k in range(1,155):
                # Imputation kNN
                imputer = KNNImputer(n_neighbors=k)
                imputed = imputer.fit_transform(data)
                data_kNNimputed = pd.DataFrame(imputed, columns=data.columns)
                # Données qui vont permettre l'entraînement et le test du modèle
                X = data_kNNimputed.drop(attribute, axis=1)
                Y = data_kNNimputed[attribute]
                # Fixer le random state permet d'obtenir les mêmes valeurs peu importe le nombre d'exécutions
                X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
                # Mis en place du modèle sur les données imputées
                model = RandomForestRegressor()
                model.fit(X_train, y_train)
                predict = model.predict(X_test)
                # Calcul du RMSE
                error = mean_squared_error(y_test, predict)
                res.append({'K': k, 'RMSE': error})
        
        return res
  

In [172]:
kNNimputation_RMSE = []
for attribute in missing_attributes:
    kNNimputation_RMSE.append(eval_kNNimputation(attribute))


{'K': 154, 'RMSE': 0.0016012047592208918}

In [173]:

# Fonction permettant d'obtenir la liste des attributs fortement corrélés à attribut_name 
# On recherche au minimum un attribut fortement corrélé.
# Le paramètre count_missing_attributes est un booléen permettant de considérer (ou non)
# les autres attributs manquants
def correlated_attributes(attribute_name):
    coef = 0.9
    # Dataframe contenant les attributs fortement corrélés
    data_corr = data_noclass.loc[:, data_noclass.corr()[attribute_name] > coef]
    
    data_corr = data_corr.drop(missing_attributes, axis=1, errors='ignore')
    while data_corr.empty:
        coef -= 0.1
        data_corr = data_noclass.loc[:, data_noclass.corr()[attribute_name] > coef]
        data_corr = data_corr.drop(missing_attributes, axis=1, errors='ignore')

    res_list = list(data_corr.columns)

    return res_list,coef

In [174]:
for attribute in missing_attributes:
    print("ADM : ",attribute,"\n"," liste des attributs corrélés : ",correlated_attributes(attribute)[0],"\n",
          "     coeficient de corrélation : ",correlated_attributes(attribute)[1])

ADM :  a_IC 
  liste des attributs corrélés :  ['a_count', 'bpol'] 
      coeficient de corrélation :  0.7000000000000001
ADM :  a_heavy 
  liste des attributs corrélés :  ['a_hyd', 'CASA-', 'chi0', 'chi1', 'DCASA', 'VAdjMa', 'VDistMa', 'vdw_vol', 'zagreb'] 
      coeficient de corrélation :  0.9
ADM :  ASA- 
  liste des attributs corrélés :  ['DASA', 'PEOE_VSA_NEG', 'Weight'] 
      coeficient de corrélation :  0.9
ADM :  ASA+ 
  liste des attributs corrélés :  ['a_nH'] 
      coeficient de corrélation :  0.9
ADM :  vsurf_R 
  liste des attributs corrélés :  ['apol', 'a_nC', 'a_nH', 'chi0v_C', 'chi0_C', 'chi1v_C', 'chi1_C', 'mr', 'PC-', 'SlogP', 'SMR', 'std_dim2'] 
      coeficient de corrélation :  0.10000000000000014
ADM :  vsurf_S 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C'] 
      coeficient de corrélation :  0.40000000000000013
ADM :  vsurf_V 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C'] 
      coeficient de corrélation :  0.40000000000000013


Nous allons maintenant procéder à la régression linéaire de chaque ADM, avec visualisation des données tests en fonction des données prédites. De plus, nous avons calculé le coefficient de détermination, donnant la qualité de prédiction de la régression linéaire, et les valeurs d'erreurs absolues moyennes que l'on comparera à celle de l'imputation.

In [175]:
# Liste contenant les erreurs absolues moyennes
linearR_MAE = []
# Résultats de la régression linéaires
linearR_results = []

In [176]:
data_impLinear = data

def linearRegression_imputation():
    for attribute in missing_attributes:
        # Dataframe des données sans les lignes aux valeurs manquantes dans pour la colonne étudiée
        data_noNaN = data_noclass[~data_noclass[attribute].isna()]
        # Dataframe contenant les données d'entrainement
        X = data_noNaN.drop(attribute,axis=1)
        # Datframe de l'attribut cible
        Y = data_noNaN[attribute]
        # Création du modèle de régression
        # On lui donne en données d'etraînement toutes les données qui ne sont pas nulles
        model = LinearRegression().fit(X = X,y = Y)
        # Il faut qu'on prédise les données avec les valeurs manquantes
        predict = model.predict(data_noclass.drop(attribute))

        data_impLinear[attribute].fillna(predict)


        # Calcul de l'erreur absolue moyenne et sauvegarde dans la liste dédiée
        MAE = metrics.mean_absolute_error(y_test, predict)
        linearR_MAE.append(round(MAE*100,3))
        r2_score = model.score(X, Y)
        res = [list(y_test),list(predict),r2_score]
        linearR_results.append(res)


# Exécution de la fonction                      
linearRegression_imputation()

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:

data_impLinear.isna
#observation des résultats de la régression linéaire
def visualisation_linearRegression():
    i =1
    fig = make_subplots(rows=2, cols=4,subplot_titles=missing_attributes)
    for ADM in linearR_results:
        coef = round(ADM[2]*100,4)
        if i <= 4 :
            fig.add_trace(
            go.Scatter(x=ADM[0], y=ADM[1],mode="markers",name=coef),
            row=1, col=i
            )
        
        else :
            fig.add_trace(
            go.Scatter(x=ADM[0], y=ADM[1],mode="markers",name=coef),
            row=2, col=i%4)
        i+=1
    
    fig.update_layout(height=600, width=1000, title_text="Résulats de la régression linéaire des ADM",legend_title="Coefficients de corrélation en %")
    fig.show()

visualisation_linearRegression()

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ASA+
- ASA-
- CASA+
- CASA-
- DASA
- ...


<bound method NDFrame.describe of          apol      ASA+      ASA-   a_count  a_donacc   a_heavy     a_hyd  \
0    0.684213  0.035935  0.999003  1.000000      0.00  1.000000  1.000000   
1    0.164036  0.422997  0.067557  0.193548      0.00  0.000000  0.117647   
2    0.730622  0.086779  0.753228  0.322581      0.00  0.363636  0.470588   
3    0.605576  0.107740  0.597048  0.354839      0.25  0.272727  0.294118   
4    0.687723  0.136829  0.715166  0.322581      0.00  0.333333  0.441176   
..        ...       ...       ...       ...       ...       ...       ...   
149  0.516128  0.286831  0.493783  0.322581      0.00  0.212121  0.323529   
150  0.562859  0.189654  0.608770  0.322581      0.00  0.272727  0.323529   
151  0.401424  0.477292  0.111778  0.387097      0.00  0.121212  0.235294   
152  0.458158  0.276278  0.421494  0.290323      0.00  0.212121  0.235294   
153  0.501215  0.490079  0.112692  0.451613      0.00  0.181818  0.294118   

         a_IC      a_nC  a_nCl  ...      

In [ ]:
kNN_imputation_res = []
def kNN_imputation_MAE():
    for attribute in missing_attributes:
        y_true = data_noclass[attribute].dropna().values
        y_imputed = data_kNN[attribute].values
        MAE = metrics.mean_absolute_error(y_true, y_imputed)
        kNN_imputation_MAE.append(round(MAE*100,3))

kNN_imputation_MAE()

ValueError: Found input variables with inconsistent numbers of samples: [153, 154]

Nous allons maintenant observer les différences d'erreurs moyennes absolues entre l'imputation par régression linéaire et k-NN.

In [ ]:


fig = go.Figure(layout_title_text="Comparaison des erreurs absolues moyennes entre l'imputaion par régression linéaire et l'imputation kNN")
fig.add_trace(go.Histogram(histfunc="min", y=linearR_MAE, x=missing_attributes, name="Imputation ar régression linéaire",marker_color="#B784B7"))
fig.add_trace(go.Histogram(histfunc="min", y=kNN_imputation_MAE, x=missing_attributes, name="Imputation kNN",marker_color="#E6A4B4"))
fig.show()

ValueError: 
    Invalid value of type 'builtins.function' received for the 'y' property of histogram
        Received value: <function kNN_imputation_MAE at 0x0000020DDA57C9A0>

    The 'y' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

On en déduit donc, d'après les résultats précédents, qu'il est plus cohérent de considérer les autres ADM lors de l'imputation par régression linéaire. Cependant, en observant la liste des attributs corrélés pour chaque ADM, on se rend compte ue pour la meilleure corrélation possible, certains ADM dépendant d'autres ADM.

ADM :  a_IC 
  liste des attributs corrélés :  ['a_count', 'a_heavy', 'bpol', 'CASA+', 'chi0', 'chi1', 'diameter', 'PC+', 'PEOE_VSA_PNEG', 'PEOE_VSA_POL', 'radius', 'std_dim1', 'TPSA', 'VDistEq', 'VDistMa', 'vdw_vol', 'VSA']
ADM :  a_heavy 
  liste des attributs corrélés :  ['a_hyd', 'CASA-', 'chi0', 'chi1', 'DCASA', 'VAdjMa', 'VDistMa', 'vdw_vol', 'zagreb']
ADM :  ASA- 
  liste des attributs corrélés :  ['DASA', 'PEOE_VSA_NEG', 'Weight']
ADM :  ASA+ 
  liste des attributs corrélés :  ['a_nH', 'CASA+', 'chi0v_C', 'chi0_C', 'chi1v_C', 'chi1_C', 'std_dim2', 'VAdjEq']
ADM :  vsurf_R 
  liste des attributs corrélés :  ['apol', 'ASA+', 'a_IC', 'a_nC', 'a_nH', 'chi0v_C', 'chi0_C', 'chi1v_C', 'chi1_C', 'mr', 'PC-', 'SlogP', 'SMR', 'std_dim2', 'vsurf_S']
ADM :  vsurf_S 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C', 'vsurf_V']
ADM :  vsurf_V 
  liste des attributs corrélés :  ['chi1v_C', 'chi1_C', 'vsurf_S']


Création d'une fonction permettant de remplir les données manquantes d'un attribut donné en paramètres, en utilisant une régression linéaire stochastique. PQ ?
Réalisée à l'aide de la référence  4.

a_IC


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

b)

c)

2 - Mesures de distance

a)

b)

3 - Choix du modèle de classification

b)

4 - Application

In [ ]:
#okokoibn
def fun():
    return 897

### **Références**

1. https://stefvanbuuren.name/fimd/sec-MCAR.html
2. https://medium.com/analytics-vidhya/different-types-of-missing-data-59c87c046bf7
3. https://www.datacamp.com/tutorial/techniques-to-handle-missing-data-values
4. https://www.kaggle.com/code/shashankasubrahmanya/missing-data-imputation-using-regression
5. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3489534/#:%7E:text=We%20suggest%20that%20normalization%20be,imputation%2C%20significance%20analysis%20and%20visualization